In [1]:
import threading
import queue
import time

##### Declaring the number of nodes

In [2]:
n = 12

##### Declaring Node class

In [3]:
class Node:
    def __init__(self, uid, in_queue, prev_queue, next_queue):
        self.uid = uid
        self.status = 'unknown'
        self._in_queue = in_queue
        self._prev_queue = prev_queue
        self._next_queue = next_queue

    def send_prev(self, data):
        self._prev_queue.put(data)

    def send_next(self, data):
        self._next_queue.put(data)

    def accept(self):
        data = self._in_queue.get()
        self._in_queue.task_done()
        return data

#### LCR algorithm

In [4]:
def lcr_node_worker(node):
    u = node.uid
    send = node.uid

    while True:
        node.send_next(send)

        v = node.accept()
        if v == "TERM":
            break

        if v is None:
            send = v
            continue
        if v > u:
            send = v
            node.status = 'non-leader'
        elif v < u:
            send = None
        else:
            node.status = 'leader'
            return

In [5]:
def lcr_run(total_nodes):
    queues = [queue.Queue() for _ in range(total_nodes)]
    prev_queues = [queues[(i - 1) % total_nodes] for i in range(total_nodes)]
    next_queues = [queues[(i + 1) % total_nodes] for i in range(total_nodes)]
    nodes = [Node(i, queues[i], prev_queues[i], next_queues[i]) for i in range(total_nodes)]

    threads = []
    for node in nodes:
        t = threading.Thread(target=lcr_node_worker, args=(node,))
        t.start()
        threads.append(t)

    time.sleep(3)
    for q in queues:
        q.put("TERM")

    for t in threads:
        t.join()

    for node in nodes:
        print(f"{node.uid}: {node.status}")
    print("\nLCR algorithm complete.")

In [6]:
lcr_run(n)

0: non-leader
1: non-leader
2: non-leader
3: non-leader
4: non-leader
5: non-leader
6: non-leader
7: non-leader
8: non-leader
9: non-leader
10: non-leader
11: leader

LCR algorithm complete.


#### HS algorithm

In [7]:
def hs_node_worker(node):
    u = node.uid
    send_plus = ('from_prev', u, 'out', 1)
    send_minus = ('from_next', u, 'out', 1)
    phase = 0

    while True:
        node.send_next(send_plus)
        node.send_next(send_minus)

        v_prev, v_next = None, None
        while v_prev is None or v_next is None:
            v = node.accept()
            if v == "TERM":
                return
            if v is None:
                continue
            if v[0] == 'from_prev':
                v_prev = v[1:]
            if v[0] == 'from_next':
                v_next = v[1:]

        if v_prev[1] == 'out':
            v, out, h = v_prev
            if v > u and h > 1:
                send_plus = ('from_prev', v, 'out', h - 1)
            elif v > u and h == 1:
                send_minus = ('from_next', v, 'in', 1)
            elif v == u:
                node.status = 'leader'

        if v_next[1] == 'out':
            v, out, h = v_next
            if v > u and h > 1:
                send_minus = ('from_next', v, 'out', h - 1)
            elif v > u and h == 1:
                send_plus = ('from_prev', v, 'in', 1)
            elif v == u:
                node.status = 'leader'

        if v_prev[1] == 'in' and v_prev[2] == 1 and v_prev[0] != u:
            v, in_, h = v_prev
            send_plus = ('from_prev', v, 'in', 1)
        if v_next[1] == 'in' and v_next[2] == 1 and v_next[0] != u:
            v, in_, h = v_next
            send_minus = ('from_next', v, 'in', 1)
        if ((v_prev[0] == u and v_prev[1] == 'in' and v_prev[2] == 1)
                and (v_next[0] == u and v_next[1] == 'in' and v_next[2] == 1)):
            phase += 1
            send_plus = ('from_prev', u, 'out', 2 ** phase)
            send_minus = ('from_next', u, 'out', 2 ** phase)

In [8]:
def hs_run(total_nodes):
    queues = [queue.Queue() for _ in range(total_nodes)]
    prev_queues = [queues[(i - 1) % total_nodes] for i in range(total_nodes)]
    next_queues = [queues[(i + 1) % total_nodes] for i in range(total_nodes)]
    nodes = [Node(i, queues[i], prev_queues[i], next_queues[i]) for i in range(total_nodes)]

    threads = []
    for node in nodes:
        t = threading.Thread(target=hs_node_worker, args=(node,))
        t.start()
        threads.append(t)

    time.sleep(3)
    for q in queues:
        q.put("TERM")

    for t in threads:
        t.join()

    for node in nodes:
        print(f"{node.uid}: {node.status}")
    print("\nHS algorithm complete.")

In [9]:
hs_run(n)

0: unknown
1: unknown
2: unknown
3: unknown
4: unknown
5: unknown
6: unknown
7: unknown
8: unknown
9: unknown
10: unknown
11: leader

HS algorithm complete.
